In [1]:
import pickle
with open('df.pickle', mode='rb') as h:
    df_train, df_test = pickle.load(h)
with open('hex.pickle', mode='rb') as h:
    D = len(pickle.load(h)['id_name'])  # number of labels

In [2]:
import cv2
import numpy as np
from os.path import join
img_dir = '../pascal12/JPEGImages'
img_mean = np.load('../ilsvrc_2012_mean.npy').astype(np.float32)  # 3*256*256
img_mean = np.swapaxes(np.swapaxes(img_mean, 0, 1), 1, 2)  # convert to XY[BGR]
def load_image(filename):
    """
    Loads image, subtract mean, resize to 227*227. No axis rotation or normalization.
    Returns:
        Result image as an np.float32 array with size 227*227*3. Values within [0, 255].
    """
    img = cv2.resize(cv2.imread(join(img_dir, filename)), (256, 256))
    return cv2.resize(img.astype(np.float32) - img_mean, (227, 227))

In [3]:
import h5py
def save_XY(filename, X, Y=None):
    with h5py.File('../{}.h5'.format(filename), mode='w') as h:
        h.create_dataset('data', data=X)
        if Y is not None:
            h.create_dataset('label', data=Y)

In [4]:
X_train = map(load_image, df_train['img'])
X_train = np.swapaxes(np.swapaxes(X_train, 2, 3), 1, 2)  # convert to [BGR]XY for caffe classifier. Other classifiers needs to convert back
N = len(X_train)  # size of training data
Y_train = np.zeros((N, D), dtype=np.float32)
pseudo_labels = df_train['pseudo_label']
for i in range(0, N):
    Y_train[i, pseudo_labels[i]] = 1.0 / len(pseudo_labels[i])  # for softmax + cross entropy loss
save_XY('train', X_train, Y_train)
save_XY('test', map(load_image, df_test['img']))  # axis order XY[BGR]